In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


**Generating input files for pyclone**

In [2]:
import os 
os.chdir("/Desktop/working_folder/")

In [3]:
def pyclone(y,z,t):
    x1=data.filter(regex= y)
    x2=data.filter(regex= z)
    x3=data.filter(regex="SNV|Gene|sex")
    x=pd.concat([x3,x1,x2],axis=1)    
    x["mutation_id"]= x["Gene"]+ ":"+ x["SNV"]
    x.drop(["SNV","Gene"],axis=1,inplace=True )
    x.columns=["sex","var_counts","total_counts","major_cn","mutation_id"]
    x["normal_cn"]=int(2)
    x["minor_cn"]=0
    x["ref_counts"]=x["total_counts"]-x["var_counts"]
    x["Chromosome"]=x.mutation_id.str.split(":").apply(lambda x :x[1])
    x["Chromosome"]=x.Chromosome.str.split(".").apply(lambda x :x[0])
    x["Chromosome"]=x.Chromosome.str.replace("chr","")
    x["sex_chr"]=x["sex"]+ ":"+ x["Chromosome"]
    x["normal_cn"]=x['sex_chr'].apply(lambda x: int(1) if x == "M:X" else int(2))
    x=x[[ 'mutation_id', 'ref_counts','var_counts','normal_cn', 'minor_cn','major_cn'
       ]]
    x.dropna(inplace=True)
    x=x.astype(int, errors='ignore')
    os.chdir("Desktop/working_folder/Pyclone_inputs/")
    x.to_csv(t, sep = '\t', index=False)  

In [4]:
for filename in os.listdir(): 
    if filename.endswith("input.txt"):
            os.chdir("/Desktop/working_folder/")
            data=pd.read_csv(filename,delimiter="\t")
            data.columns = data.columns.str.replace('(\.\d+)$','') # rename the auto-renaming . so duplicate columsn can be removed
            data = data.loc[:, ~data.columns.duplicated()]
            df2 = data.filter(regex='_CNV-Int')
            num = 1+len(df2.columns)
            for i in range(1,num):
                y="nA"+str(i)
                z="A"+str(i)+"_CNV-Int"
                t=filename.split('_')[0]+"_A"+str(i)+".tsv"
                pyclone(y,z,t)
